In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import requests
import json
import re
import time
import datetime
import os
from datetime import tzinfo
from dateutil.tz import tzoffset
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
import time
import getpass
import numpy as np

##### Instructions : Download Appropriate Web Driver from the ChromeDrivers folder and add the Executeable to Path

In [ ]:
'''
if (platform.system() == "Windows"):
    driver = webdriver.Chrome(executable_path=os.path.abspath('ChromeDrivers/chromedriver_windows.exe'))
elif (platform.system() == "Darwin"):
    driver = webdriver.Chrome('ChromeDrivers/chromedriver_mac.exe')
elif (platform.system() == "Linux"):
    driver = webdriver.Chrome(executable_path='ChromeDrivers/chromedriver_linux.exe')
driver.get("http://cape.ucsd.edu")
'''

<h3>Let's Login To CAPES</h3>

In [421]:
#enter credentials
username = ''
password = getpass.getpass()


if(len(username) < 1):
    assert len(username) > 0
    
if(len(password) < 1):  
    password = '' #safety
    assert len(password) > 0

##init chrome driver
driver = webdriver.Chrome()
driver.get("https://cape.ucsd.edu/responses/Results.aspx")

#send data
elem = driver.find_element_by_name("urn:mace:ucsd.edu:sso:username")
elem.clear()
elem.send_keys(username)

elem = driver.find_element_by_name("urn:mace:ucsd.edu:sso:password")
elem.clear()
elem.send_keys(password)
    
elem = driver.find_element_by_name("_eventId_proceed").click()

#reset username & password for safety
username = ''
password = ''

········


In [372]:
select = Select(driver.find_element_by_name('ctl00$ContentPlaceHolder1$ddlDepartments'))
options = select.options
del options[0] #remove "select department tag"
departments = [o.get_attribute('value') for o in options]

### Download All Page Sources For Each Department

In [374]:
basecape = 'http://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber='
page_sources = []
for depts in departments:
    req = basecape + depts
    driver.get(req)
    time.sleep(1)
    page_sources.append(driver.page_source)

### Build Headers For DataFrame

In [390]:
headers = []
soup = BeautifulSoup(source, 'html.parser')
table = soup.find('table', attrs={'class':'styled'})
for th in table.find('tr').findAll('th'):
    headers.append(th.text.strip())

['Instructor', 'Course', 'Term', 'Enroll', 'Evals Made', 'Rcmnd Class', 'Rcmnd Instr', 'Study Hrs/wk', 'Avg Grade Expected', 'Avg Grade Received']


### Parse Table Containing Reviews Into DataFrame

In [376]:
dataframes = []
for source in page_sources:
    soup = BeautifulSoup(source, 'html.parser')
    table = soup.find('table', attrs={'class':'styled'})
    data = []
    for i,row in enumerate(table.findAll('tr')):
        if i==0:
            continue
        else:
            col_data = []
            for td in row.findAll('td'): 
                col_data.append(td.text.strip())
        data.append(col_data)
    dataframes.append(pd.DataFrame(data))

### Concat DataFrame, Add Headers, Clean, And Save To File

In [385]:
df = pd.concat(dataframes)
df.columns = headers

In [393]:
df.reset_index(drop=True,inplace=True)

In [3]:
df.to_csv('capeReviews.csv')

NameError: name 'df' is not defined

### Second Round Cleanup

In [6]:
df = pd.read_csv(open('capeReviews.csv'), header=0)
df.drop(columns=['Unnamed: 0'],inplace=True)

In [7]:
def cleanPercentage(percent):
    percent = percent.strip()
    percent = percent.split(' ')[0]
    percent = float(percent) / 100.0
    return percent
    

In [8]:
def cleanGrades(grades):
    if(pd.notnull(grades)):
        grades.strip()
        grades = grades.split(' ')[1]
        grades = grades.strip('()')
        return float(grades)
    else:
        return grades

In [9]:
df["Rcmnd Class"] = df["Rcmnd Class"].apply(cleanPercentage)
df["Rcmnd Instr"] = df["Rcmnd Instr"].apply(cleanPercentage)

In [10]:
df["Avg Grade Expected"] = df["Avg Grade Expected"].apply(cleanGrades)
df["Avg Grade Received"] = df["Avg Grade Received"].apply(cleanGrades)

In [11]:
df.head()

,Instructor,Course,Term,Enroll,Evals Made,Rcmnd Class,Rcmnd Instr,Study Hrs/wk,Avg Grade Expected,Avg Grade Received
0,"Goldstein, Paul S",ANTH 101 - Foundations/Social Complexity (A),FA18,137,73,0.812,0.791,4.00,3.33,NaN
1,"Semendeferi, Ekaterini",ANTH 196A - Honors Studies in Anthropology (A),FA18,13,4,1.000,1.000,7.00,4.00,NaN
2,"Rothschild, Amy C",ANTH 21 - Race and Racisms (A),FA18,212,96,0.947,0.978,3.01,3.58,NaN
3,"Kang, Byung Chu",ANTH 21 - Race and Racisms (B),FA18,314,214,0.942,0.937,3.36,3.82,NaN
4,"Stewart, Alexander B",ANTH 23 - Debating Multiculturalism (A),FA18,407,351,0.879,0.956,3.53,3.58,NaN


### Adding Useful Data For Comparison to RateMyProfessor

In [12]:
def cleanTerms(terms):
    semester = terms[:2]
    year     = terms[2:4]
    
    if(semester == "WI"):
        return (int)(year+"0")
    
    if(semester == "SP"):
        return (int)(year+"1")
    
    if(semester == "S1" or semester == "S2" or semester == "S3" or semester == "SU"):
        return (int)(year+"2")
    
    if(semester == "FA"):
        return (int)(year+"3")
    

In [13]:
def splitDepartment(course):
    course = course.strip()
    course = course.split(" ")[0]
    return course

In [14]:
def splitFirstName(inst):
    if(pd.notnull(inst)):
        inst = inst.strip()
        inst = inst.split(",")[1].strip()
        inst = inst.split(" ")[0].strip()
        return inst
    else:
        return inst

In [15]:
def splitLastName(inst):
    if(pd.notnull(inst)):
        inst = inst.strip()
        inst = inst.split(",")[0].strip()
        return inst
    else:
        return inst

In [16]:
df["Term"] = df["Term"].apply(cleanTerms)

In [17]:
df["tDept"] = df["Course"].apply(splitDepartment)

In [18]:
df["tLname"] = df["Instructor"].apply(splitLastName)

In [19]:
df["tFname"] = df["Instructor"].apply(splitFirstName)

In [20]:
df["Eval / Enroll"] = df["Evals Made"].values / df["Enroll"].values

In [21]:
df = df[['Instructor','tLname','tFname','tDept','Course','Term','Enroll','Evals Made', 'Eval / Enroll',
'Rcmnd Class', 'Rcmnd Instr', 'Study Hrs/wk', 'Avg Grade Expected', 'Avg Grade Received']]

In [22]:
df.head()

,Instructor,tLname,tFname,tDept,Course,Term,Enroll,Evals Made,Eval / Enroll,Rcmnd Class,Rcmnd Instr,Study Hrs/wk,Avg Grade Expected,Avg Grade Received
0,"Goldstein, Paul S",Goldstein,Paul,ANTH,ANTH 101 - Foundations/Social Complexity (A),183,137,73,0.532847,0.812,0.791,4.00,3.33,NaN
1,"Semendeferi, Ekaterini",Semendeferi,Ekaterini,ANTH,ANTH 196A - Honors Studies in Anthropology (A),183,13,4,0.307692,1.000,1.000,7.00,4.00,NaN
2,"Rothschild, Amy C",Rothschild,Amy,ANTH,ANTH 21 - Race and Racisms (A),183,212,96,0.452830,0.947,0.978,3.01,3.58,NaN
3,"Kang, Byung Chu",Kang,Byung,ANTH,ANTH 21 - Race and Racisms (B),183,314,214,0.681529,0.942,0.937,3.36,3.82,NaN
4,"Stewart, Alexander B",Stewart,Alexander,ANTH,ANTH 23 - Debating Multiculturalism (A),183,407,351,0.862408,0.879,0.956,3.53,3.58,NaN


In [241]:
df.to_csv('capeReviewsCleaned.csv')

In [5]:
df_rmp = pd.read_csv(open('modifiedProfInfo.csv'), header=0)

### Finding Duplicate Names As This Could Cause Issues

In [24]:
fn = [(splitFirstName(s) + " " + splitLastName(s)) for s in df['Instructor'].unique() if pd.notnull(s)]

In [25]:
seen = {}
dupes = []

for x in fn:
    if x not in seen:
        seen[x] = 1
    else:
        if seen[x] == 1:
            dupes.append(x)
        seen[x] += 1
        
print("There are {} professors with the same first and last name, however they are in different departments after further analysis".format(len(dupes))) 

There are 38 professors with the same first and last name, however they are in different departments after further analysis


### Below Are the Number of Duplicate Professors In Different Departments
#### Through Individual Analysis we Deduced that the Rest of the 32 Professors were Not Actually Duplicates but Inputted Wrong in CAPES

In [26]:
dps = []
for d in dupes:
    ff = d.split(' ')[0]
    ll = d.split(' ')[1]
    if(len(df[(df['tLname'] == ll) & (df['tFname'] == ff)]['tDept'].unique()) > 1):
        dps.append((ff,ll))
display(dps)

[('Michael', 'Cole'),
 ('Dorothy', 'Yang'),
 ('David', 'Miller'),
 ('Laura', 'Stevens'),
 ('Maria', 'Cesena'),
 ('Traci', 'Voyles'),
 ('James', 'Williams'),
 ('Grant', 'Allen'),
 ('Scott', 'Vandehey'),
 ('Kefei', 'Zhang'),
 ('Jeffrey', 'Kaiser'),
 ('Dong', 'Lee'),
 ('Daniel', 'Smith'),
 ('Dong', 'Kim'),
 ('Andrew', 'Allen'),
 ('Mark', 'Young')]

### Now we will try to append the TIDS from RMP to CAPES

In [27]:
tids_nf = []
tids = df_rmp['tid'].unique()
for tid in tids:
    if(pd.notnull(tid)):
        rmp_fname = df_rmp[df_rmp['tid'] == tid]['tFname'].unique()[0]
        rmp_lname = df_rmp[df_rmp['tid'] == tid]['tLname'].unique()[0]
        rmp_fl = (rmp_fname,rmp_lname)
        if(rmp_fl not in dps):
            indices = df[(df['tLname'] == rmp_lname) & (df['tFname'] == rmp_fname)].index
            if(len(indices) > 0):
                for i in indices:
                    df.loc[i,"tid"] = tid
            else:
                tids_nf.append(tid)       

#### replace tids with int

In [69]:
df['tid'] = df['tid'].fillna(-1)
df['tid'] = df['tid'].astype(int)
df['tid'] = df['tid'].replace('-1', np.nan)

In [41]:
profs_not_on_RMP = len(df[pd.isnull(df['tid'])]['Instructor'].unique())
total_profs_on_CAPES = len(df['Instructor'].unique())
print("There are {} professors without reviews on RMP, out of the total {} professors on cape".format(profs_not_on_RMP,total_profs_on_CAPES))

There are 1820 professors without reviews on RMP, out of the total 3368 professors on cape


### Duplicates In RMP

In [42]:
trim_dps = []
for d in dps:
    if(len(df_rmp[(df_rmp['tLname'] == d[1]) & (df_rmp['tFname'] == d[0])]) > 0):
        trim_dps.append(d)
        
trim_dps

[('David', 'Miller'),
 ('Laura', 'Stevens'),
 ('Traci', 'Voyles'),
 ('Grant', 'Allen'),
 ('Scott', 'Vandehey'),
 ('Andrew', 'Allen'),
 ('Mark', 'Young')]

In [104]:
df.to_csv('capeReviewsCleaned2.csv')

In [103]:
df.head()

,Instructor,tLname,tFname,tDept,Course,Term,Enroll,Evals Made,Eval / Enroll,Rcmnd Class,Rcmnd Instr,Study Hrs/wk,Avg Grade Expected,Avg Grade Received,tid
0,"Goldstein, Paul S",Goldstein,Paul,ANTH,ANTH 101 - Foundations/Social Complexity (A),183,137,73,0.532847,0.812,0.791,4.00,3.33,NaN,222495
1,"Semendeferi, Ekaterini",Semendeferi,Ekaterini,ANTH,ANTH 196A - Honors Studies in Anthropology (A),183,13,4,0.307692,1.000,1.000,7.00,4.00,NaN,1171683
2,"Rothschild, Amy C",Rothschild,Amy,ANTH,ANTH 21 - Race and Racisms (A),183,212,96,0.452830,0.947,0.978,3.01,3.58,NaN,-1
3,"Kang, Byung Chu",Kang,Byung,ANTH,ANTH 21 - Race and Racisms (B),183,314,214,0.681529,0.942,0.937,3.36,3.82,NaN,2308255
4,"Stewart, Alexander B",Stewart,Alexander,ANTH,ANTH 23 - Debating Multiculturalism (A),183,407,351,0.862408,0.879,0.956,3.53,3.58,NaN,2099749
